In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## a. Data Pre-Processing

Load data

In [2]:
%%time
historical_transactions = pd.read_csv('../input_data/historical_transactions.csv')
new_merchant_transactions = pd.read_csv('../input_data/new_merchant_transactions.csv')
merchants = pd.read_csv('../input_data/merchants.csv')
train = pd.read_csv('../input_data/train.csv')
test = pd.read_csv('../input_data/test.csv')

Wall time: 1min 53s


*historical_transactions* and *new_merchant_transactions*

In [3]:
%%time
historical_transactions['authorized_flag'] = historical_transactions['authorized_flag'].apply(lambda x:1 if x == 'Y' else 0)
new_merchant_transactions['authorized_flag'] = new_merchant_transactions['authorized_flag'].apply(lambda x:1 if x == 'Y' else 0)

historical_transactions['category_1'] = historical_transactions['category_1'].apply(lambda x:1 if x == 'Y' else 0)
new_merchant_transactions['category_1'] = new_merchant_transactions['category_1'].apply(lambda x:1 if x == 'Y' else 0)

map_dict = {'A':0, 'B':1, 'C':2, 'nan': 3}
historical_transactions['category_3'] = historical_transactions['category_3'].apply(lambda x: map_dict[str(x)])
new_merchant_transactions['category_3'] = new_merchant_transactions['category_3'].apply(lambda x: map_dict[str(x)])

Wall time: 50.6 s


*merchants*

In [4]:
%%time
merchants['category_1'] = merchants['category_1'].apply(lambda x : 1 if x == 'Y' else 0)
merchants['category_4'] = merchants['category_4'].apply(lambda x : 1 if x == 'Y' else 0)
merchants['category_2'] = merchants['category_2'].fillna(0)

Wall time: 551 ms


*test*

In [5]:
# Missing values
test[test['first_active_month'].isna()]

,first_active_month,card_id,feature_1,feature_2,feature_3
11578,NaN,C_ID_c27b4f80f7,5,2,1


In [6]:
# Impute missing values here

concatenate *train* and *test*

In [7]:
%%time
train['type'] = 0
test['type'] = 1
train_test = pd.concat([train, test], ignore_index=True)

C:\Users\ChenP\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 410 ms


In [8]:
train_test.to_pickle('../processed_data/train_test.pkl')
historical_transactions.to_pickle('../processed_data/historical_transactions.pkl')
new_merchant_transactions.to_pickle('../processed_data/new_merchant_transactions.pkl')
merchants.to_pickle('../processed_data/merchants.pkl')